In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
states_ne = ['AL', 'BA', 'CE','MA', 'PB', 'PE', 'PI', 'SE', 'RN']
states_se = ['SP', 'MG', 'RJ', 'ES']
states_sul = ['RS', 'SC', 'PR']
states_ce = ['DF', 'GO', 'MT', 'MS']
states_no = ['AM', 'AP', 'TO', 'RR', 'RO' ,'AC', 'PA']

states_BR = states_ne+states_se+states_no+states_ce+states_sul

In [3]:
df_macro = pd.read_csv('./macro_saude.csv',index_col = 'Unnamed: 0')

df_macro.head()

,geocode,name_muni,name_region,code_region,name_macro,code_macro,state
0,1100015,Alta Floresta D'Oeste,Zona da Mata,11005,Cacoal,1101,RO
1,1100023,Ariquemes,Vale do Jamari,11001,Porto Velho,1102,RO
2,1100031,Cabixi,Cone Sul,11006,Cacoal,1101,RO
3,1100049,Cacoal,Café,11002,Cacoal,1101,RO
4,1100056,Cerejeiras,Cone Sul,11006,Cacoal,1101,RO


In [4]:
df_macro_tot = df_macro[['geocode', 'code_region']].groupby('code_region').count().rename(columns = {'geocode':'total_geocodes'})

df_macro_tot.head()

,total_geocodes
code_region,
11001,9
11002,6
11003,14
11004,5
11005,8


In [5]:
state = 'CE'

In [6]:
df = pd.DataFrame()
for state in states_BR: 
    
    df_ = pd.read_csv(f'./episcanner_{state}_dengue.csv.gz', index_col = 'Unnamed: 0')

    df = pd.concat([df, df_])
    
df.head()

,disease,CID10,year,geocode,muni_name,peak_week,beta,gamma,R0,total_cases,alpha,sum_res,ep_ini,ep_end,ep_dur
0,dengue,A90,2011,2708600,São Miguel dos Campos,20.194544,0.479272,0.300250,1.596245,74.477234,0.373530,1.219589,201046,201128,34
1,dengue,A90,2011,2708501,São Luís do Quitunde,29.001973,0.357709,0.302385,1.182961,79.080701,0.154664,2.328548,201046,201144,50
2,dengue,A90,2011,2704708,Marechal Deodoro,19.038388,0.432219,0.300102,1.440241,712.514200,0.305672,1.351712,201046,201131,37
3,dengue,A90,2011,2704005,Junqueiro,28.287041,0.424264,0.303031,1.400071,64.239404,0.285750,1.289925,201046,201142,48
4,dengue,A90,2011,2704500,Maragogi,23.737129,0.617729,0.303268,2.036910,527.419737,0.509060,0.679277,201104,201126,22


In [7]:
df.shape

(24369, 15)

In [8]:
df_m = df.merge(df_macro[['geocode', 'code_region']], left_on = 'geocode', right_on = 'geocode')

df_m.isnull().sum()

disease        0
CID10          0
year           0
geocode        0
muni_name      0
peak_week      0
beta           0
gamma          0
R0             0
total_cases    0
alpha          0
sum_res        0
ep_ini         0
ep_end         0
ep_dur         0
code_region    0
dtype: int64

In [9]:
df_m_1 = df_m[['year', 'code_region', 'geocode']].groupby(['year', 'code_region']).count()

df_m_1.head()

geocode
year code_region         
2011 11001              4
     11002              2
     11003              8
     11004              3
     11005              4

In [10]:
df_m.columns

Index(['disease', 'CID10', 'year', 'geocode', 'muni_name', 'peak_week', 'beta',
       'gamma', 'R0', 'total_cases', 'alpha', 'sum_res', 'ep_ini', 'ep_end',
       'ep_dur', 'code_region'],
      dtype='object')

In [11]:
df_m_2 = df_m[['year', 'code_region', 'peak_week', 'R0']].groupby(['year', 'code_region']).mean()

df_m_2.head()

peak_week        R0
year code_region                     
2011 11001        15.304109  1.648559
     11002        13.853240  1.547726
     11003        15.182029  1.547248
     11004        14.383883  1.626356
     11005         9.916184  1.852409

In [12]:
df_m_3 = df_m[['year', 'code_region', 'total_cases']].groupby(['year', 'code_region']).sum()

df_m_3.head()

total_cases
year code_region             
2011 11001         726.425015
     11002         341.364333
     11003        1148.942986
     11004        1810.718043
     11005        1149.261079

In [13]:
df_end = df_m_1.merge(df_m_2, left_index = True, right_index = True)

df_end = df_end.merge(df_m_3, left_index = True, right_index = True).reset_index()

df_end = df_end.merge(df_macro_tot, left_on = 'code_region', right_index = True)

df_end['perc_geocode'] = df_end['geocode']/df_end['total_geocodes']

df_end.head()

,year,code_region,geocode,peak_week,R0,total_cases,total_geocodes,perc_geocode
0,2011,11001,4,15.304109,1.648559,726.425015,9,0.444444
1,2011,11002,2,13.853240,1.547726,341.364333,6,0.333333
2,2011,11003,8,15.182029,1.547248,1148.942986,14,0.571429
3,2011,11004,3,14.383883,1.626356,1810.718043,5,0.600000
4,2011,11005,4,9.916184,1.852409,1149.261079,8,0.500000


In [14]:
df_end.to_csv('episcanner_regional.csv.gz', index = False)